<a href="https://colab.research.google.com/github/danisshai/IBM-DATA-SCIENCE-PROGRAM/blob/main/10_SpaceX_Capstone/3_2_Interactive_Dashboard_DASH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making a Real Time Interactive Dashboard 
## Plotly - Dash

1. You will see some screenshots and descriptions about the funtionality of the dashboard.
2. The dashboard is mean to be a website, not a Jupyter Notebook and therefor here I will only paste the code necessary to run it. 


# 1. Dashboard Description

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

- Which site has the largest successful launches?
- Which site has the highest launch success rate?
- Which payload range(s) has the highest launch success rate?
- Which payload range(s) has the lowest launch success rate?
- Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest

## The dashboard contain:
1. A dropdown menu to filter launches by site
2. A pie chart that changes acording to the dropdown menu. If all sites are selected, it show the proportioin each site contribute to all the success launches. Otherwise it will show the proportion of success (1) launches and failure launches (1) of the corresponding location.
3. A Scatterplot wich change acording to the selected site and the selected payload mass at a interactive slider. It will show you if the distribution of successes and failures of launches and its correlation with the payload mass, aditionally you may get some insight on the relationships between those two variables and the Booster Version Category.


In the following images you can observe a preview of the dashboar if you want to interact with it, please continue to the following section.



<center><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/pie-chart-all.PNG" /></center>



<center><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/pie-chart-one.PNG" /></center>



<center><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/scatter-point.PNG" /></center>


# 2. Dashboard Code

I currently do not provide a link to the dashboard. However you can run it on your own by following the next steps on your own computer.

### 1. Retrieve the data and install libraries

In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2022-05-12 22:41:56--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv.1’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2022-05-12 22:41:57 (250 MB/s) - ‘spacex_launch_dash.csv.1’ saved [2476/2476]



In [ ]:
!pip3 install pandas dash

### 2. Save the following code at a .py file and execute it with python

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

lauch_sites = ['ALL SITES']+ spacex_df['Launch Site'].unique().tolist()
# Create a dash application
app = dash.Dash(__name__)


# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(lauch_sites, lauch_sites[0], id='site-dropdown', placeholder="Select a Launch Site here",
                                searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(min_payload, max_payload, 500, 
                                value=[min_payload, max_payload],id='payload-slider'),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL SITES':
        pie_data = spacex_df.groupby(['Launch Site'])['class'].count()
        fig = px.pie(pie_data, values='class', 
            names=pie_data.index, 
            title='Total Success Launches by Site '+entered_site)
        return fig
    else:
        pie_data = spacex_df[spacex_df['Launch Site'] == entered_site]
        pie_data = pie_data.groupby("class")['class'].count()
        fig = px.pie(pie_data, values='class', 
            names=['No Succed', 'Succed'], 
            title='Total success Lauches for Site '+entered_site)
        return fig



# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='payload-slider', component_property='value'),
    Input(component_id='site-dropdown', component_property='value')
    ]
)
def get_pie_chart(silider_values,entered_site):
    if(entered_site != 'ALL SITES'):
        data = spacex_df[spacex_df['Launch Site'] == entered_site]
    else:
        data = spacex_df
    data = data[(data['Payload Mass (kg)'] >= silider_values[0]) & (data['Payload Mass (kg)'] <= silider_values[1])]
    data['class'] = data['class'].astype(str)
    fig = px.scatter(data, x='Payload Mass (kg)', y = 'class', color='Booster Version Category',
    title= 'Correlation Between Payload and Success for '+entered_site)
    fig.update_traces(marker_size=10)

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


# The following cells are only of reference for the autor to develop the dashboard

In [ ]:
import pandas as pd
spacex_df = pd.read_csv('spacex_launch_dash.csv')
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [ ]:
['All places']+ spacex_df['Launch Site'].unique().tolist()

['All places', 'CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']

In [ ]:
spacex_df.groupby("class")['class'].count()

class
0    32
1    24
Name: class, dtype: int64

In [ ]:
spacex_df.groupby(['Launch Site'])['class'].count()

Launch Site
CCAFS LC-40     26
CCAFS SLC-40     7
KSC LC-39A      13
VAFB SLC-4E     10
Name: class, dtype: int64

In [ ]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [ ]:
spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40']

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0
5,5,7,CCAFS LC-40,0,3170.0,F9 v1.1,v1.1
6,6,8,CCAFS LC-40,0,3325.0,F9 v1.1,v1.1
7,7,9,CCAFS LC-40,0,2296.0,F9 v1.1,v1.1
8,8,10,CCAFS LC-40,0,1316.0,F9 v1.1,v1.1
9,9,11,CCAFS LC-40,0,4535.0,F9 v1.1,v1.1
